In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray

data_dir = Path('../data')

In [ ]:
metrics_dir = data_dir / 'outputs/plots/metrics/x1-y1-z1/net_cdf'

def read_plot_metrics(plot_id: str):
    metrics = xr.open_dataset(metrics_dir / f"{plot_id}_with_cover.nc", decode_coords='all')
    metrics.load()
    metrics.close()
    return metrics

In [3]:
plots = gpd.read_file(data_dir / "outputs/plots/plots.geojson")
plots = plots.set_index('id')
plots

,site,plot_number,site_plot_id,geometry
id,,,,
AGG_O_01_P1,AGG_O_01,1,AGG_O_01_P1,"POLYGON ((463042.83 5259846.736, 463025.797 52..."
AGG_O_01_P2,AGG_O_01,2,AGG_O_01_P2,"POLYGON ((463124.556 5259819.234, 463116.068 5..."
AGG_O_01_P3,AGG_O_01,3,AGG_O_01_P3,"POLYGON ((463201.174 5259815.806, 463200.551 5..."
AGG_O_01_P4,AGG_O_01,4,AGG_O_01_P4,"POLYGON ((463257.777 5259801.962, 463245.303 5..."
AGG_O_01_P5,AGG_O_01,5,AGG_O_01_P5,"POLYGON ((463303.022 5259789.552, 463289.794 5..."
...,...,...,...,...
ULY_Y_96_P1,ULY_Y_96,1,ULY_Y_96_P1,"POLYGON ((455348.476 5275834.517, 455348.154 5..."
ULY_Y_96_P2,ULY_Y_96,2,ULY_Y_96_P2,"POLYGON ((455296.576 5275837.224, 455299.223 5..."
ULY_Y_96_P3,ULY_Y_96,3,ULY_Y_96_P3,"POLYGON ((455252.092 5275834.16, 455248.849 52..."


In [63]:
def create_plot_summary(row: gpd.GeoSeries) -> pd.Series:
    id = row.name
    metrics : xr.Dataset = read_plot_metrics(id)

    mean_metrics_names = [
        "point_density",
        "pulse_density",
        "scan_angle_mean",
        "chm",
        "veg_height_mean",
        "veg_height_median",
        "crr",
        "veg_height_q10",
        "veg_height_q20",
        "veg_height_q30",
        "veg_height_q40",
        "veg_height_q50",
        "veg_height_q60",
        "veg_height_q70",
        "veg_height_q80",
        "veg_height_q90",
        "veg_height_sd",
        'veg_height_cv',
        'veg_height_skew',
        'veg_height_kurt',
        'veg_height_gini',
        'canopy_cover_gt1m',
        'fhd',
        'vci',
        'shann_capture',
    ]
    # Skip point and pulse density and scan angle
    sd_metric_names = mean_metrics_names[3:]
    # CV for all the ones that are in height units
    cv_metric_names = [
        "chm",
        "veg_height_mean",
        "veg_height_median",
        "veg_height_q10",
        "veg_height_q20",
        "veg_height_q30",
        "veg_height_q40",
        "veg_height_q50",
        "veg_height_q60",
        "veg_height_q70",
        "veg_height_q80",
        "veg_height_q90",
        "veg_height_sd",
    ]
    
    # done separately as they may not exist
    cover_metrics = [
        'groundstorey_capture',
        'understorey_capture',
        'midstorey_capture',
        'upperstorey_capture'
    ]

    mean_metrics : pd.Series = metrics[mean_metrics_names].mean(dim=['x', 'y']).to_pandas()
    sd_metrics : pd.Series = metrics[sd_metric_names].std(dim=['x', 'y']).to_pandas()
    cv_metrics : pd.Series = (sd_metrics[cv_metric_names] / mean_metrics[cv_metric_names])
    mean_metrics = mean_metrics.add_prefix('mean__')
    sd_metrics = sd_metrics.add_prefix('sd__')
    cv_metrics = cv_metrics.add_prefix('cv__')

    # I only want max of chm
    max_metrics = pd.Series({
        "max__chm": metrics['chm'].max(dim=['x', 'y']).item()
    })

    mean_cover_metrics = pd.Series({
        f"mean__{cm}": metrics[cm].mean(dim=['x', 'y']).item() for cm in cover_metrics if cm in metrics
    })

    sd_cover_metrics = pd.Series({
        f"sd__{cm}": metrics[cm].std(dim=['x', 'y']).item() for cm in cover_metrics if cm in metrics
    })

    plot_summary_metrics = pd.concat([mean_metrics, max_metrics, sd_metrics, cv_metrics, mean_cover_metrics, sd_cover_metrics])
    plot_summary_metrics.name = id

    return plot_summary_metrics

In [64]:
create_plot_summary(plots.iloc[0])

mean__point_density          94.731332
mean__pulse_density          48.682927
mean__scan_angle_mean        -0.070185
mean__chm                     9.104318
mean__veg_height_mean         5.137496
                               ...    
mean__understorey_capture     0.158089
mean__upperstorey_capture     0.480691
sd__groundstorey_capture      0.290380
sd__understorey_capture       0.144361
sd__upperstorey_capture       0.215257
Name: AGG_O_01_P1, Length: 67, dtype: float64

In [67]:
plot_summaries = plots.apply(create_plot_summary, axis=1)
plot_summaries

,cv__chm,cv__veg_height_mean,cv__veg_height_median,cv__veg_height_q10,cv__veg_height_q20,cv__veg_height_q30,cv__veg_height_q40,cv__veg_height_q50,cv__veg_height_q60,cv__veg_height_q70,...,sd__veg_height_q20,sd__veg_height_q30,sd__veg_height_q40,sd__veg_height_q50,sd__veg_height_q60,sd__veg_height_q70,sd__veg_height_q80,sd__veg_height_q90,sd__veg_height_sd,sd__veg_height_skew
id,,,,,,,,,,,,,,,,,,,,,
AGG_O_01_P1,0.333808,0.425354,0.489945,1.739751,1.115931,0.768326,0.581273,0.489945,0.440378,0.407512,...,2.337715,2.772345,2.886210,2.911427,2.905377,2.907627,2.898930,2.901512,1.078531,0.870518
AGG_O_01_P2,0.321928,0.399944,0.441722,1.482898,0.960439,0.662018,0.511847,0.441722,0.403030,0.376795,...,2.478124,2.724146,2.741132,2.749006,2.754852,2.760448,2.761082,2.788032,0.996851,0.772531
AGG_O_01_P3,0.270166,0.375165,0.445083,1.499740,0.982958,0.700272,0.537327,0.445083,0.389831,0.354417,...,2.691013,3.138907,3.211832,3.161811,3.087255,3.033358,2.970419,2.907858,1.092759,0.886172
AGG_O_01_P4,0.728379,0.829358,0.995413,1.501746,1.328288,1.180890,1.073566,0.995413,0.929683,0.879785,...,7.489644,10.099930,12.401368,14.118188,15.462890,16.621882,17.584190,18.333689,6.892083,1.181471
AGG_O_01_P5,0.591712,0.676299,0.782689,1.232634,1.005016,0.896340,0.830547,0.782689,0.740013,0.705013,...,10.024234,11.926630,13.812166,15.401384,16.736571,17.904529,18.682749,19.363676,6.788184,1.086882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ULY_Y_96_P1,0.663666,0.830295,0.997172,1.538833,1.343570,1.195296,1.080600,0.997172,0.927448,0.863658,...,3.203405,3.596331,4.000175,4.323214,4.633587,4.905537,5.136496,5.307063,1.894300,1.282089
ULY_Y_96_P2,1.059070,1.303202,1.629176,2.373361,2.077025,1.947170,1.774145,1.629176,1.528335,1.423188,...,2.180661,3.175736,3.964667,4.811867,5.466306,5.977312,6.393802,6.721609,2.597684,1.180520
ULY_Y_96_P3,0.985092,1.043863,1.406755,1.178660,1.093558,1.206692,1.340269,1.406755,1.362587,1.311276,...,0.738959,1.082485,1.571437,2.165252,2.649559,3.187197,3.532450,4.019286,1.631394,1.100349


In [69]:
plot_summary_dir = data_dir / 'outputs'
plot_summaries.to_csv(plot_summary_dir / 'plot_summary_metrics.csv')